In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('default')

sentiment_df = pd.read_csv('/mnt/data/fear_greed_index.csv')
trader_df = pd.read_csv('/mnt/data/historical_data.csv')

In [6]:
sentiment_df.head(), trader_df.head()

(    timestamp  value classification        date
 0  1517463000     30           Fear  2018-02-01
 1  1517549400     15   Extreme Fear  2018-02-02
 2  1517635800     40           Fear  2018-02-03
 3  1517722200     24   Extreme Fear  2018-02-04
 4  1517808600     11   Extreme Fear  2018-02-05,
                                       Account  Coin  Execution Price  \
 0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
 1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
 2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
 3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
 4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   
 
    Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
 0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
 1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
 2       144.09   1150.63  BUY 

In [7]:
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
trader_df['time'] = pd.to_datetime(trader_df['Timestamp IST'], format='%d-%m-%Y %H:%M')
trader_df['Date'] = trader_df['time'].dt.date
trader_df['Date'] = pd.to_datetime(trader_df['Date'])


sentiment_df['classification'] = sentiment_df['classification'].str.lower()


merged_df = pd.merge(
    trader_df,
    sentiment_df,
    left_on='Date',
    right_on='date',
    how='inner'
)

In [8]:
import os
output_directory = 'csv_files'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

merged_df.to_csv(os.path.join(output_directory, 'merged_data.csv'), index=False)


merged_df['profit'] = merged_df['Closed PnL']
merged_df['is_profitable'] = merged_df['profit'] > 0


merged_df['trade_volume'] = abs(merged_df['Size Tokens'] * merged_df['Execution Price'])
merged_df['risk_exposure'] = merged_df['trade_volume'] * merged_df['leverage']

sentiment_summary = merged_df.groupby('classification').agg({
    'profit': ['mean', 'sum'],
    'leverage': 'mean',
    'trade_volume': 'mean',
    'risk_exposure': 'mean'
}).reset_index()

sentiment_summary.columns = ['Sentiment','Avg_Profit','Total_Profit',
                             'Avg_Leverage','Avg_Volume','Avg_Risk']

sentiment_summary.to_csv('csv_files/sentiment_summary.csv', index=False)


display(sentiment_summary)

KeyError: 'leverage'

In [ ]:
merged_df['leverage'] = 1

In [ ]:
print(merged_df.columns)

In [ ]:
merged_df.info()

In [ ]:
import os

output_directory_plots = 'outputs'
if not os.path.exists(output_directory_plots):
    os.makedirs(output_directory_plots)

plt.figure(figsize=(6,4))
sns.boxplot(x='classification', y='profit', data=merged_df)
plt.title('Profit Distribution: Fear vs Greed')
plt.savefig(os.path.join(output_directory_plots, 'pnl_vs_sentiment.png'))
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.barplot(x='classification', y='leverage', data=merged_df)
plt.title('Average Leverage by Market Sentiment')
plt.savefig('outputs/leverage_vs_sentiment.png')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.barplot(x='classification', y='trade_volume', data=merged_df)
plt.title('Trade Volume by Market Sentiment')
plt.savefig('outputs/volume_vs_sentiment.png')
plt.show()

In [ ]:
merged_df['win'] = merged_df['profit'] > 0


merged_df['risk_adjusted_pnl'] = merged_df['profit'] / (merged_df['risk_exposure'] + 1)


merged_df['aggression_score'] = merged_df['leverage'] * abs(merged_df['Size Tokens'])

behavior_profile = merged_df.groupby('classification').agg(
    avg_profit=('profit','mean'),
    win_rate=('win','mean'),
    avg_leverage=('leverage','mean'),
    avg_risk_adj=('risk_adjusted_pnl','mean'),
    aggression=('aggression_score','mean')
).reset_index()

behavior_profile.to_csv('csv_files/trader_behavior_profile.csv', index=False)
behavior_profile

In [ ]:
plt.figure(figsize=(6,4))
sns.barplot(x='classification', y='risk_adjusted_pnl', data=merged_df)
plt.title('Risk-Adjusted Profit by Market Sentiment')
plt.savefig('outputs/risk_adjusted_returns.png')
plt.show()